In [1]:
import tensorflow as tf
from tqdm import tqdm, trange
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.models import load_model, Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Softmax, Input, Reshape,Lambda,\
    Conv2D, MaxPool2D, Dropout,Concatenate,Embedding,BatchNormalization,TimeDistributed,LSTM
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import cv2
import re
import numpy as np
import os
import time
import json
from tensorflow.keras.applications.inception_v3 import  InceptionV3,preprocess_input
from tqdm import tqdm, trange
import math
from utils import batch_generator
# from NIC import model,greedy_inference_model,image_dense_lstm,text_emb_lstm
from pickle import dump, load

In [2]:
fp3= "data/captions_train2014.json"
with open(fp3,'r') as f3:
    mscoco_data= json.load(f3)
    

all_img_name_vector=[]
all_captions=[]
PATH_DIR= '/home/yiyang/project/caption/caption'
PATH_TRAIN=PATH_DIR + '/data/train2014/'
for annot in mscoco_data['annotations']:
    caption =  annot['caption']
   
    caption = caption.rstrip()
    caption = caption.lower()
    caption = [x.replace(".", "") for x in caption]
    caption = "".join(caption)
    caption = caption.rstrip()
    caption = "".join(caption)
    
    image_id = annot['image_id']
    full_coco_image_path = PATH_TRAIN + 'COCO_train2014_' + '%012d.jpg' % (image_id)

    all_img_name_vector.append(full_coco_image_path)
    all_captions.append(caption)
#     
# train_captions, img_name_vector = shuffle(all_captions,
#                                           all_img_name_vector,
#                                           random_state=1)
# num_examples = 400000
# train_captions = all_captions[:num_examples]
# image_vec = all_img_name_vector[:num_examples]
print(len(all_captions))
print(len(all_img_name_vector))
num_examples=len(all_captions)

414113
414113


In [3]:
fp4= "sentence_dict2.json"
with open(fp4,"r") as f4 :
    sentiment_dict= json.load(f4)
    
image_vec=[]
sentiment_vec=[]
caption_vec=[]
error=0
for i in range(num_examples):
    cap= all_captions[i]
    if cap in sentiment_dict.keys():
        senti= sentiment_dict[cap]
        
    else:
        senti=0
        error+=1
        continue
        # print(cap)
    cap= '<start> ' + cap +' <end>'
    caption_vec.append(cap)
    sentiment_vec.append(senti)
    image_vec.append(all_img_name_vector[i])
    
print(error)
print(len(caption_vec))
print(len(image_vec))


26819
387294
387294


In [4]:
num_examples=387294
image_name_train= image_vec[:num_examples]
senti_train= sentiment_vec[:num_examples]
caption_train= caption_vec[:num_examples]
print(caption_train[0])


<start> a very clean and well decorated empty bathroom <end>


In [5]:
def load_image(image_path):
    img= cv2.imread(image_path)
    img= cv2.resize(img,(299,299))
    img = (img*1.0)/255
    img = img-0.5 
    img = img*2
    return img






In [6]:
fp2="data/senticap_dataset.json"

with open(fp2,'r') as f2:
    senticap_data= json.load(f2)
senticap_list=[]
senticap_list_raw=[]
l=0
for img_obj in senticap_data['images']:
    # if (l>20):
    #     break
    # l+=1
    img_name= img_obj['filename']
    for sent in img_obj['sentences']:
        sentiment= sent['sentiment']
        raw= sent['raw'].lower()
        w0= raw.rstrip()
        w0 = [x.replace(".", "") for x in w0]
        w0= [x.replace(",","") for x in w0]
        w0= [x.replace("'","") for x in w0]
        w0= [x.replace("?","") for x in w0]
        w0="".join(w0)
        senticap_list.append((img_name,sentiment,w0))
        senticap_list_raw.append((img_name,w0))


        

In [7]:
"""read from output of nlp treebank"""

fname="new_out.txt"
i=0
l1=10000
senti_score=[]
senti_avg=[]
error=0
senti_var=[]
train_caption=[]
senti_cap_dict={}
with open(fname) as f1:
    while(i<l1):
        w1= f1.readline().strip()
        if not w1:
            break
        w2= f1.readline().strip()
        w1 = [x.replace(".", "") for x in w1]
        w1= [x.replace(",","") for x in w1]
        w1= [x.replace("'","") for x in w1]
        w1= [x.replace("?","") for x in w1]
        w1= "".join(x for x in w1)
        w1= w1.rstrip()
        w1= "".join(x for x in w1)

        if w2.islower():
            w1=w2
            w2=f1.readline().strip()
        a= 0
        if w2=="Negative":
            a= -1
        elif w2=="Positive":
            a= 1
        elif w2=="Very negative":
            a= -1
        elif w2=="Very positive":
            a= 1
        elif w2=="Neutral":
            a= 0
        else:
            a=0
            error+=1
            print(w2)
        senti_score.append(a)
        train_caption.append(w1)
        senti_cap_dict[w1]=a
        i+=1
# print(senti_score[0:500] )
print(len(train_caption),"train-cap_length")
print(len(senti_score),"train-senti_score")


" .
8861 train-cap_length
8861 train-senti_score


In [8]:
PATH_DIR= '/home/yiyang/project/caption/caption'
PATH= PATH_DIR+'/data/val2014/'
# image_vec=[]
# sentiment_vec=[]
# caption_vec=[]

for item in senticap_list:
    img= item[0]
    if item[1]==1:
        senti=1
    elif item[1]==0:
        senti=-1
    else:
        senti=0

    caption= item[2]
    # print(caption+"#")
    if caption in senti_cap_dict.keys():
        new_senti= senti_cap_dict[caption]
    else:
        new_senti= 0
    img= PATH + img 
    caption= '<start> ' + caption + ' <end>'
    image_vec.append(img)
    sentiment_vec.append(new_senti)
    caption_vec.append(caption)
    
print(len(image_vec),"length of combined dataset")
print(len(caption_vec))
print(len(sentiment_vec))

396142 length of combined dataset
396142
396142


In [9]:
units= 512
embedding_size=511
BATCH_SIZE=80
vocab_size=10000
max_length=20
num_examples=396080
reg=1e-4

In [10]:
t=396142
image_name_train= image_vec[t-num_examples:]
senti_train= sentiment_vec[t-num_examples:]
caption_train= caption_vec[t-num_examples:]


In [11]:
vocab_size=10000
top_k = vocab_size-1
tokenizer = keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(caption_vec)
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'


train_seqs = tokenizer.texts_to_sequences(caption_train)
cap_vector_train = keras.preprocessing.sequence.pad_sequences(train_seqs,dtype='int32', padding='post',truncating='post',maxlen=max_length+1,value=0)

print(caption_train[0])
print(cap_vector_train[0])



<start> a surfer rides his bike down the street <end>
[  3   2 356 372  38 226  31   7  24   4   0   0   0   0   0   0   0   0
   0   0   0]


In [23]:
print(len(image_name_train))

396080


In [16]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [31]:
weights_path = 'data/image_net.h5'

In [32]:
class Image_model(tf.keras.Model):
    
    def __init__(self):
        super(Image_model, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.image_model= InceptionV3(include_top=True,weights=None)
        self.image_model.load_weights(weights_path)
        for layer in self.image_model.layers[:312]:
            layer.trainable = False
        self.model_input= self.image_model.input
        self.model_output= self.image_model.get_layer('avg_pool').output
        self.new_model=tf.keras.Model(self.model_input,self.model_output)
    def call(self,x):
        output= self.new_model(x)
        return output
    

In [33]:
def load_image2(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path


In [34]:
image_features_extract_model = Image_model()


In [35]:
encode_train = sorted(set(image_vec))

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image2, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)
for img, path in tqdm(image_dataset):
    batch_features = image_features_extract_model.predict(x=img, steps=16)
    # batch_features = image_features_extract_model(img)
      # batch_features = tf.reshape(batch_features,
      #                             (batch_features.shape[0], -1, batch_features.shape[3]))
    batch_features=np.reshape(batch_features,(-1,batch_features.shape[1]))
    # break
    # np.save(path,batch_features)

    for bf, p in tqdm(zip(batch_features, path)):
        path_of_feature = p.numpy().decode("utf-8")
        # path_of_feature = p
        # break
        np.save(path_of_feature, bf)
        # break
#         

0it [00:00, ?it/s]

0it [00:00, ?it/s]

16it [00:00, 1557.92it/s]


1it [00:05,  5.55s/it]

0it [00:00, ?it/s]

...


5311it [50:25,  1.75it/s]

0it [00:00, ?it/s]

7it [00:00, 3279.73it/s]


5312it [50:26,  1.47it/s]

5312it [50:26,  1.76it/s]

In [12]:
class RNN_train(tf.keras.Model):
    def __init__(self,vocab_size, max_len, units, img_embedding_size):
        super(RNN_train, self).__init__()
        # self.image_model= Image_model()
        self.vocab_size=vocab_size
        self.max_len= max_len
        self.units=units
        # self.reg= reg
        self.image_embedding_layer= Dense(img_embedding_size, use_bias=False, name = 'emb_img')
        self.seq_embedding_layer=Embedding(vocab_size, units, mask_zero = False, name = 'emb_text')
        self.BN_layer= BatchNormalization(name='batch_normalization')
        self.expand_dim= Lambda(lambda x : K.expand_dims(x, axis=1))
        # self.state_c= tf.zeros()
        self.lstm_layer=  LSTM(self.units, return_sequences = True, return_state = True, dropout=0.2, name = 'lstm')
        self.softmax_layer= Dense(vocab_size, activation='softmax')
        self.time_distribute= TimeDistributed(self.softmax_layer)
        self.argmax_layer=  Lambda(lambda x : (K.argmax(x)))
        
    def call(self, inputs):
        # image_input = Input(shape=(2048)
        # 
        # seq_input = Input(shape=(self.max_len,))
        # senti_input = Input(shape=(1,))
        # state_c = Input(shape=(self.units,))
        # state_h = Input(shape=(self.units,))
        
        image_input = inputs[0]
        
        seq_input = inputs[1]
        senti_input = inputs[2]
        state_c = inputs[3]
        state_h = inputs[4]
        # tf.print(senti_input)
        
        # x= CNN_encoder(image_input)
        # tf.print(x)
        x= Dropout(0.2)(image_input)
        x= self.image_embedding_layer(x)
        senti_input= K.cast(senti_input,dtype=tf.float32)
        # tf.print(senti_input)
        x= Concatenate(axis=-1)([senti_input, x])
        x= self.BN_layer(x)
        x= self.expand_dim(x)
        
        x_seq= self.seq_embedding_layer(seq_input)
        # tf.print(x_seq.shape,"embed_seq")
        x_seq= Dropout(0.2)(x_seq)
        
        _, init_state_c, init_state_h = self.lstm_layer(x, initial_state=[state_c, state_h])
        
        out, _, _ = self.lstm_layer(x_seq, initial_state=[init_state_c, init_state_h])
        # tf.print(out.shape,"lstm_out")
        output= self.time_distribute(out)
        
        # train_model= Model(inputs=[image_input,seq_input,senti_input, state_c, state_h], outputs=output)
        return output
        
    def inference_call(self, inputs):
        image_input = inputs[0]
        
        seq_input = inputs[1]
        senti_input = inputs[2]
        state_c0 = inputs[3]
        state_h0 = inputs[4]
        x= image_input
        # tf.print(x)

        x= self.image_embedding_layer(x)
        # tf.print(x.shape,"emb_img")
        x_senti= K.cast(senti_input,dtype=tf.float32)
        x_senti= tf.expand_dims(x_senti,axis=-1)
        # tf.print(senti_input)
        x= Concatenate(axis=-1)([x_senti, x])
        x= self.BN_layer(x)
        x= self.expand_dim(x)
        # tf.print(x.shape,"img_input")
        x_seq= self.seq_embedding_layer(seq_input)
        x_seq= tf.expand_dims(x_seq,axis=1)
        # tf.print(x_seq.shape,"emb_seq")
        # tf.print(state_c0.shape,"c0")
        o, state_c, state_h = self.lstm_layer(x, initial_state=[state_c0, state_h0])
        # tf.print(state_c.shape,"init_c")
        # tf.print(o.shape,"img_out_shape")
        outputs = []

        for i in range(self.max_len):

            out, state_c, state_h = self.lstm_layer(x_seq, initial_state=[state_c,state_h])
            # tf.print(state_c.shape,"c")
            
            output = self.softmax_layer(state_c)
            # tf.print(output.shape,"softmax_output_shape")
            # outputs.append(output)
            x_seq = self.argmax_layer(output)
            outputs.append(x_seq)
            x_seq= K.expand_dims(x_seq)
            # tf.print(x_seq.shape,"argmax")
            x_seq = self.seq_embedding_layer(x_seq)
            # tf.print(x_seq.shape,"new_seq")
            
        return outputs
        
        
        


In [24]:
reg=1e-4
decay=0
lr=0.0001
initial_epoch=0
embedding_size=511

# NIC_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr = lr, decay=decay), metrics=['accuracy'])

rnn_train_model= RNN_train(vocab_size,max_length,units,embedding_size)
rnn_train_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr = lr, decay=decay), metrics=['accuracy'])

In [29]:

# print(caption_train[0].shape)
start=0
train_num=396080
generator_train= batch_generator(BATCH_SIZE,vocab_size,train_num,start,image_name_train,senti_train,cap_vector_train,)


In [30]:
start2=0
train_num2=2000
generator_dev= batch_generator(BATCH_SIZE,vocab_size,train_num2,start2,image_name_train,senti_train,cap_vector_train,)


In [41]:
# tf.config.experimental_run_functions_eagerly(False)
BATCH_SIZE=80
tf.print(lr)
rnn_train_model.load_weights("models/10kmodel0_20kdata.h5")
rnn_train_model.fit_generator(generator_train, steps_per_epoch=train_num//BATCH_SIZE,
                        epochs=4,
                        verbose=1, 
                        initial_epoch = 3)

rnn_train_model.save_weights("models/10kmodel1_400kdata.h5")

0.0001


Epoch 4/4


   1/4951 [..............................] - ETA: 22:52 - loss: 2.5669 - accuracy: 0.6019

   2/4951 [..............................] - ETA: 27:43 - loss: 2.4322 - accuracy: 0.6159

   3/4951 [..............................] - ETA: 23:59 - loss: 2.4203 - accuracy: 0.6181

...

2521/4951 [==============>...............] - ETA: 9:02 - loss: 1.7647 - accuracy: 0.6748

2522/4951 [==============>...............] - ETA: 9:02 - loss: 1.7646 - accuracy: 0.6748

2523/4951 [==============>...............] - ETA: 9:02 - loss: 1.7646 - accuracy: 0.6748

2524/4951 [==============>...............] - ETA: 9:01 - loss: 1.7645 - accuracy: 0.6748

2525/4951 [==============>...............] - ETA: 9:01 - loss: 1.7644 - accuracy: 0.6748

In [29]:
rnn_train_model.save_weights("models/10kmodel1_400kdata.h5")
# rnn_train_model.load_weights("models/10kmodel0_200kdata.h5")


In [27]:

# rnn_train_model.load_weights("models/10kmodel0.h5")
start=0
train_num=20000
generator_train= batch_generator(BATCH_SIZE,vocab_size,train_num,start,image_name_train,senti_train,cap_vector_train,)

rnn_train_model.fit_generator(generator_train, steps_per_epoch=train_num//BATCH_SIZE,
                        epochs=1,
                        verbose=1, 
                        initial_epoch = 0)
rnn_train_model.load_weights("models/10kmodel1_400kdata.h5")





  1/250 [..............................] - ETA: 3:06 - loss: 9.2107 - accuracy: 0.0000e+00

  2/250 [..............................] - ETA: 1:55 - loss: 9.2088 - accuracy: 0.0000e+00

  3/250 [..............................] - ETA: 1:29 - loss: 9.2069 - accuracy: 0.0153    

  4/250 [..............................] - ETA: 1:15 - loss: 9.2052 - accuracy: 0.0404

  5/250 [..............................] - ETA: 1:07 - loss: 9.2033 - accuracy: 0.0658

...

246/250 [============================>.] - ETA: 0s - loss: 4.8141 - accuracy: 0.3923

247/250 [============================>.] - ETA: 0s - loss: 4.8091 - accuracy: 0.3924

248/250 [============================>.] - ETA: 0s - loss: 4.8031 - accuracy: 0.3926

249/250 [============================>.] - ETA: 0s - loss: 4.7960 - accuracy: 0.3927

250/250 [==============================] - 39s 155ms/step - loss: 4.7897 - accuracy: 0.3929


In [37]:
test_imgs=[]
num_test=2000
start=0
for j in tqdm(range(start,start+num_test)):
    img= np.load( image_name_train[j]+'.npy')
    test_imgs.append(img)
test_imgs= np.array(test_imgs,dtype=np.int)
test_sentis= np.array(senti_train[start:start+num_test],dtype=np.float32)
test_sequences= np.array(cap_vector_train[start:start+num_test])
# test_seq= np.ones(shape=(500,1))
# test_seq= 3* test_seq
# test_out=[]
print(test_imgs.shape)
tf.print(test_imgs[0].shape)
tf.print(test_imgs[0])

  0%|          | 0/2000 [00:00<?, ?it/s]

 33%|███▎      | 661/2000 [00:00<00:00, 6601.15it/s]

 67%|██████▋   | 1338/2000 [00:00<00:00, 6648.85it/s]

 93%|█████████▎| 1861/2000 [00:00<00:00, 6147.44it/s]

100%|██████████| 2000/2000 [00:00<00:00, 6216.79it/s]

(2000, 2048)
(2048,)


array([0, 0, 0, ..., 0, 0, 0])


In [38]:
N = test_imgs.shape[0]

startseq = np.repeat([tokenizer.word_index['<start>']], N)
a0 = np.zeros([N, units],dtype=np.float32)
c0 = np.zeros([N, units],dtype=np.float32)
# test_sentis= np.ones([num_test])
# test_sentis= - np.ones([num_test])
# test_sentis= np.zeros([num_test])

# output dims: [32, N, 7378]
test_num=N
img_input= tf.convert_to_tensor(test_imgs[:test_num])
seq_input= tf.convert_to_tensor(startseq[:test_num])
senti_input= tf.convert_to_tensor(test_sentis[:test_num])
state_a0= tf.convert_to_tensor(a0[:test_num])
state_c0= tf.convert_to_tensor(c0[:test_num])
y_preds= rnn_train_model.inference_call([img_input,seq_input,senti_input,state_a0,state_c0])
y_preds= np.array(y_preds)

y_preds = np.transpose(y_preds, axes = [1,0])
# print(y_preds)
# y_preds = np.array(rnn_train_model.inference_call([test_imgs[:80], startseq[:80],test_sentis[:80], a0[:80], c0[:80]]))
# y_preds = RNN_inf(test_imgs[:10], startseq[:10],test_sentis[:10], a0[:10], c0[:10])
    
# output dims: [N, 32, 7378]
# y_preds = np.transpose(y_preds, axes = [1,0,2])
# 
# sequences = np.argmax(y_preds, axis = -1)
sequences=y_preds
sents = tokenizer.sequences_to_texts(sequences)
# print(sents[450:])
# 
# # if post_process:
#     # post processing: 'endseq'
sents_pp = []
for sent in sents:
    if '<end>' in sent.split():
        words = sent.split()
        sents_pp.append(' '.join(words[:words.index('<end>')]))
    else:
        sents_pp.append(sent)
sents = sents_pp


In [39]:
print(sents[:10])

['a close up of a very nice looking looking side', 'a refrigerator with a large fridge and sink', 'a close up of a very pretty blue and white clock', 'a close up of a very nice looking school clock tower', 'a beautiful woman is standing in a kitchen with a toothbrush', 'a close up of a very nice looking school clock tower', 'a little boy is playing with a toy in his hand', 'a close up of a very pretty blue and white photo of a beautiful blue sky', 'a close up of a large clock tower with a large sky in the background', 'a close up of a very pretty blue and white photo of a beautiful blue sky']


In [40]:
# f1='output/gen_data.json'
# f2='output/gen_data.txt'
gen_data={}
gen_data['annotations']=[]
ref_data={}
ref_data['annotations']=[]
start=0
for i in trange(test_num):
    img_id= image_name_train[start+i]
    gen_cap= sents[i]
    ref_cap= caption_train[start+i]
    ref_cap= ref_cap.replace("<start>","")
    ref_cap= ref_cap.replace("<end>","")
    ref_cap= ref_cap.strip()
    ref_cap= "".join(ref_cap)
    gen_data['annotations'].append({'image_id':img_id,'caption':gen_cap})
    
    ref_data['annotations'].append({'image_id':img_id,'caption':ref_cap})
    
with open("output/gen_data.json", "w") as f2:
    json.dump(gen_data, f2)
    
with open("output/ref_data.json", "w") as f3:
    json.dump(ref_data, f3)

with open("output/gen_data_ori.txt","w") as f4:
    for item in gen_data["annotations"]:
        cap= item['caption']
        f4.write("%s .\n" % cap)

  0%|          | 0/2000 [00:00<?, ?it/s]

100%|██████████| 2000/2000 [00:00<00:00, 244851.37it/s]

In [30]:

img_num=2
 
print(image_name_train[img_num])
# img=load_image(image_name_train[0])
img=cv2.imread(image_name_train[img_num]) 
print(caption_train[img_num])
print(senti_train[img_num])
# img=cv2.imshow('image',img)
cv2.imshow('ImageWindow', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

 


/home/yiyang/project/caption/caption/data/train2014/COCO_train2014_000000281631.jpg
<start> a clock in middle of two building below blue sky <end>
0


In [66]:
# # @tf.function
# def RNN_inf(image_input, seq_input, senti_input, state_c, state_h):
#     # image_input = Input(2048,)        
#     # seq_input = Input(1,)
#     # senti_input = Input(1,)
#     # state_c = Input(units)
#     # state_h = Input(units)
#     # tf.print(senti_input)
#     
#     # x= image_input
#     # tf.print(x.shape)
#     # tf.print(image_input.dtype)
#     # tf.print(seq_input.dtype)
#     # tf.print(senti_input.dtype)
#     # tf.print(state_c.dtype)
#     # tf.print(state_h.dtype)
#     
#     x= rnn_train_model.image_embedding_layer(image_input)
#     # tf.print(x.shape)
#     x_senti= K.cast(senti_input,dtype=tf.float32)
#     x_senti= tf.expand_dims(x_senti,axis=-1)
#     # tf.print(senti_input.shape,"senti_input_expand")
#     x= Concatenate(axis=-1)([x_senti, x])
#     # tf.print(x.shape)
#     x= rnn_train_model.BN_layer(x)
#     x= rnn_train_model.expand_dim(x)
#     # tf.print(seq_input.shape,"seq_input")
#     x_seq= tf.expand_dims(seq_input,axis=-1)
#     x_seq= rnn_train_model.seq_embedding_layer(x_seq)
#     
#     # tf.print(x_seq.shape,"embed seq")
#     _, init_state_c, init_state_h = rnn_train_model.lstm_layer(x, initial_state=[state_c, state_h])
#     # tf.print(init_state_c.shape,"state_c")
#     # out, _, _ = self.lstm_layer(x_seq, initial_state=[init_state_c, init_state_h])
#     # output= self.time_distribute(out)
#     
#     outputs = []
#     argmax_layer=  Lambda(lambda x : K.argmax(x))
#     
#     for i in range(rnn_train_model.max_len):
# 
#         out, state_c, state_h = rnn_train_model.lstm_layer(x_seq, initial_state=[init_state_c,init_state_h])
#         tf.print(out.shape,"lstm_out")
#         tf.print(state_c.shape,"state_c")
#         init_state_c= state_c 
#         init_state_h= state_h
#         output = rnn_train_model.softmax_layer(out)
#         # tf.print(output.shape,"softmax output")
#         outputs.append(output)
#         x_seq = argmax_layer(output)
#         # tf.print(x_seq.shape)
#         x_seq = rnn_train_model.seq_embedding_layer(x_seq)
#         # tf.print(x_seq.shape)
#             
#     return outputs